Prerequisite packages

In [ ]:
import os,shutil,pickle,tqdm,sys,random,re,string,pause, datetime,glob
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# # The GPU id to use, usually either "0" or "1"
# os.environ["CUDA_VISIBLE_DEVICES"]="1" 
import keras
import sent2vec
import seq2seq
from seq2seq.models import AttentionSeq2Seq
from seq2seq.models import Seq2Seq

import tensorflow as tf
import tensorboard as tb

import numpy as np
import pandas as pd
import tqdm
from tqdm import tqdm
from random import shuffle
from math import log, floor

from keras.utils import multi_gpu_model

# from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
from keras import *
from keras.engine.topology import *
from keras.optimizers import *
from keras.metrics import *

import gensim
from gensim.models.word2vec import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *

from sklearn.model_selection import *
from sklearn.decomposition import *
from sklearn.cluster import *
from sklearn.metrics.pairwise import *
from sklearn.neighbors import KNeighborsClassifier
# from collections import Counter
from keras.utils.generic_utils import *
from keras import regularizers
import unicodedata as udata
from keras.applications import *
from keras.preprocessing.image import *

from keras import backend 
from imblearn.ensemble import *
from imblearn.combine import *
# from python.keras import backend 
# Embedding(10,20)
from keras_transformer.extras import ReusableEmbedding, TiedOutputEmbedding
from keras_transformer.position import TransformerCoordinateEmbedding
from keras_transformer.transformer import TransformerACT, TransformerBlock
from keras_transformer.bert import (
    BatchGeneratorForBERT, masked_perplexity,
    MaskedPenalizedSparseCategoricalCrossentropy)

import keras_metrics as km
from keras_trans_mask import RemoveMask, RestoreMask

from keras_multi_head import *
from keras_self_attention import SeqSelfAttention

from sklearn.metrics.pairwise import *
from sklearn.model_selection import *
from sklearn.manifold import *
from sklearn.decomposition import *
from sklearn.cluster import *
from sklearn import preprocessing

from sklearn.metrics import *
from sklearn.preprocessing import MultiLabelBinarizer
from collections import Counter

In [ ]:
#neede files in same dir.
from models import transformer_bert_model
from bpe import BPEEncoder
from utils import *

load data

In [ ]:
root_dir = './data/tree-rep-profiles_o2o/'
train_pid_paths = pickle.load(open(root_dir + 'train_pid_paths.pkl','rb'))
valid_pid_paths = pickle.load(open(root_dir + 'valid_pid_paths.pkl','rb'))
test_pid_paths = pickle.load(open(root_dir + 'test_pid_paths.pkl','rb'))
exp_pid_paths = pickle.load(open(root_dir + 'exp_pid_paths.pkl','rb'))
# o2o_pid_paths = pickle.load(open(root_dir + 'o2o_pid_paths.pkl','rb'))

train_emb_api,train_emb , train_rep_ans = pickle.load(open(root_dir + 'pids_train.pkl','rb'))
valid_emb_api,valid_emb, valid_rep_ans = pickle.load(open(root_dir + 'pids_valid.pkl','rb'))
test_emb_api,test_emb ,test_rep_ans = pickle.load(open(root_dir + 'pids_test.pkl','rb'))
# o2o_api,o2o_emb,o2o_rep = pickle.load(open(root_dir + 'pids_o2o.pkl','rb'))
exp_api,exp_emb = pickle.load(open(root_dir + 'pids_exp.pkl','rb'))
emb_matrix = pickle.load(open('data/api_emb_matrix.pkl','rb'))

In [ ]:
def basic_statistics(all_length):
    '''
    input: length list of elements
    output1: mean、std、mode、min、q1、median(q2)、q3、max、iqr、outlier、far out
    output2: statistics graph、10%~90% form
    '''
    stat_dict = {}
    stat_dict['mean'] = np.mean(all_length)
    stat_dict['std'] = np.std(all_length)
    stat_dict['mode'] = np.argmax(np.bincount(all_length))
    stat_dict['min'] = np.min(all_length)
    stat_dict['q1'] = np.quantile(all_length,0.25)
    stat_dict['median'] = np.quantile(all_length,0.5)
    stat_dict['q3'] = np.quantile(all_length,0.75)
    stat_dict['max'] = np.max(all_length)
    stat_dict['iqr'] = stat_dict['q3'] - stat_dict['q1']
    stat_dict['outlier'] = stat_dict['q3'] + 1.5*stat_dict['iqr']
    stat_dict['far_out'] = stat_dict['q3'] + 3*stat_dict['iqr']
    for i in [10,20,30,40,50,60,70,80,90,100]:
        stat_dict[str(i)+'%'] = np.percentile(all_length,i)
    return pd.DataFrame.from_dict(stat_dict,orient='index',columns=['length'])

load trained NN model

In [ ]:
def bin_acc(y_true, y_pred):
    return binary_accuracy(y_true, y_pred)

In [ ]:
model_save_path = './model/o2o_stage_gru_selfatt/XXX.h5' #訓練好的模型
model = load_model(model_save_path+'_all.h5',custom_objects={'bin_acc': bin_acc, 'SeqSelfAttention': SeqSelfAttention})
# model = load_model(model_save_path+'_all.h5',custom_objects={'bin_acc': bin_acc,'RemoveMask':RemoveMask,'RestoreMask':RestoreMask})#, 'SeqSelfAttention': SeqSelfAttention}) #如果上面的load失敗的話，可以用此
model.load_weights(model_save_path)
model.summary()
thr = 0.51 #0.482 #設定0004.於驗證資料集中所得到最佳的threshold

In [ ]:
emb_model = Model(inputs=model.inputs,  outputs=model.layers[5].output) 
#.layers[5].中的數字可能要調整，讓emb_model最後一層剛好是最終的embedding相加(Add)
emb_model.load_weights(model_save_path,by_name=True)
emb_model.summary()

# Representative execution pattern vector
* 一個profile所有important call invocation於向量空間中的表示法

In [ ]:
# 要記得先手動更改以下參數!! (依據不同dataset load近來的參數名稱會不同)
def avg_train_vector(root_dir,thr,train_pid_paths=exp_pid_paths,train_emb_api=exp_api,
                     train_emb=exp_emb,loner=False):
    '''
    train_pid_paths: 每一筆profile路徑位置
    train_emb_api: api embedding vectors
    train_emb: SentVec embedding vectors
    loner: False=>train/dev/test Set ; True: Loner Set
    '''
    no_ind_rep = 0
    no_ind_rep_li = []
    tree_vec = []
    empty_hash_li = []
    problem_hash_li = []
    un_aliase_path = './data/tree-rep-profiles_o2o/EXP_rev/'
    if not os.path.exists(root_dir):
        os.makedirs(root_dir,exist_ok=True)
    for path,api,emb in tqdm(zip(train_pid_paths,train_emb_api,train_emb)): #tqdm
        if loner:
            if path not in (claim_path + unk_path):
                continue
            recent_fam = path.split('/')[-3] #fakeav_fakealeart
            recent_hash = path.split('/')[-1].split('_')[0]
            if '_' in recent_fam:
                all_path_un_aliase = glob.glob(un_aliase_path+'*/*/'+current_hash+'*')
                recent_fam = all_path_un_aliase[0].split('/')[4]
                
        else:
            recent_fam = path.split('/')[-3].split('_')[0]
        recent_tree = path.split('/')[-2]
        hash_id = path.split('/')[-1][:5]
        pid_id = path.split('/')[-1].split('_')[-1].split('.')[0][-3:]
        with open(path,encoding='ISO 8859-1') as f: #X2
            lines = f.read()
        lines = re.sub(r'[^\x00-\x7F]+','', lines)
        lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
        lines = re.sub('=\\n','',lines)
        lines = re.sub('y\\n','',lines)
        lines = lines.splitlines()
        length = len(lines)
        if loner:
            if length < 2:
                empty_hash_li.append(path)
                continue
        else:
            if length < 11:
                print('No common rep. Should not in a tree:',path)
                problem_hash_li.append(path)
                continue
        try:
            assert api[:length][-1] != 0
            assert api[:length+1][-1] == 0
        except AssertionError:
            print('Mask assertion ERR:',path)
            continue
        api_e = np.expand_dims(api, axis=0)
        emb_e = np.expand_dims(emb, axis=0)
        final_emb_e = emb_model.predict([api_e,emb_e]) #(1,length,768)
        final_emb = final_emb_e[0][:length] #(length,768)
        byte_rep_e = model.predict([api_e,emb_e])
        byte_rep = byte_rep_e[0][:length] # (length,1)
        byte_rep_thr = []
        for num in byte_rep:
            if num[0] < thr:
                byte_rep_thr.append(0)
            else:
                byte_rep_thr.append(1) # high than threshold, keep it
        byte_rep_thr = np.array(byte_rep_thr)
        byte_rep_thr = np.expand_dims(byte_rep_thr,axis=-1)
        mul_emb = np.multiply(final_emb,byte_rep_thr)
        rep_emb = [] #超過threshold的emb才留下來
        for emb in mul_emb:
            if (sum(emb) <0) or (sum(emb)>0):
                rep_emb.append(emb)
        rep_emb_avg = np.average(rep_emb,axis=0) #把自己各api的embedding取平均，應該一定要有，不然就代表這個profile不該屬於這棵tree 
        try:
            if not (rep_emb_avg <0 or rep_emb_avg>0):
                no_ind_rep+=1
                no_ind_rep_li.append(path)
                continue #沒有individual REP
        except ValueError: #正常profile
            pass
        rep_emb_avg = np.expand_dims(rep_emb_avg,axis=0)
        pickle.dump(file=open(root_dir+recent_fam+'_'+recent_tree+'_'+hash_id+'_'+pid_id+'.pkl','wb'),obj=rep_emb_avg)
    print('No ind REP#:',no_ind_rep)
    if loner:
        return no_ind_rep_li,empty_hash_li,problem_hash_li
    return no_ind_rep_li

產生train/valid/test的r，用以下，要更改root_dir、及上述function的輸入變數

In [ ]:
# train/valid/test要改三次跑三次
root_dir = './data/tree-rep-profiles_o2o/trainingProfile_REP/' #最終輸出r的資料夾，會自動創立資料夾，存放pkl，記得最後的斜線
no_ind_rep = avg_train_vector(root_dir=root_dir,thr=thr,loner=False)
with open(root_dir + 'no_rep_file.txt','w') as fp:
    for path in no_ind_rep:
        fp.write(str(path.split('/')[-3:])+'\n') #no rep的profile path紀錄檔

產生loner的r，用以下，要更改root_dir(輸出r的資料夾)、及上述function的輸入變數

In [ ]:
# loner
loner_dir = './data/tree-rep-profiles_o2o/lonerProfile_REP/'
no_ind_rep,empty_files,problem_files = avg_train_vector(root_dir=loner_dir,thr=0.482,loner=True)
with open(loner_dir + 'no_rep_file.txt','w') as fp:
    for path in no_ind_rep:
        fp.write(str(path.split('/')[-1:])+'\n') #-no rep的profile path紀錄檔
with open(loner_dir + 'empty_file.txt','w') as fp:
    for path in empty_files:
        fp.write(str(path.split('/')[-1:])+'\n') #空的或太短的profile
with open(loner_dir + 'error_file.txt','w') as fp:
    for path in problem_files:
        fp.write(str(path.split('/')[-1:])+'\n') #有問題的profile(assertion failed)